<a href="https://colab.research.google.com/github/Waseem9211/sentiment-analysis/blob/master/medical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import *
from keras.models import Model
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import backend as K
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [3]:
df_train=pd.read_csv('/content/train_F3WbcTw.csv')
df_test=pd.read_csv('/content/test_tOlRoBf.csv')
print("Training data size:",df_train.shape)
print("Test data size:",df_test.shape)
df_train.head()

Training data size: (5279, 4)
Test data size: (2924, 3)


,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2
2,fe809672251f6bd0d986e00380f48d047c7e7b76,Interesting that it only targets S1P-1/5 recep...,fingolimod,2
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,"Very interesting, grand merci. Now I wonder wh...",ocrevus,2
4,b227688381f9b25e5b65109dd00f7f895e838249,"Hi everybody, My latest MRI results for Brain ...",gilenya,1


In [4]:
df_train['sentiment'].value_counts()

2    3825
1     837
0     617
Name: sentiment, dtype: int64

In [5]:
df_train['drug'].value_counts().head(10)

ocrevus        676
gilenya        666
ocrelizumab    441
entyvio        303
humira         270
fingolimod     238
remicade       229
opdivo         224
tarceva        218
cladribine     200
Name: drug, dtype: int64

In [6]:

BAD_SYMBOLS_RE = re.compile('[^a-zA-Z0-9]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() 
    text=text.strip()
    no_coms=re.sub(r'\.com','',text)
    no_urls=re.sub('https?://www','',no_coms)
    no_urls1=re.sub('https?://','',no_urls)     
    text = BAD_SYMBOLS_RE.sub(' ', no_urls1) 
    #text = re.sub('\d+','',text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

df_train['text'] = df_train['text'].apply(clean_text)
df_test['text'] = df_test['text'].apply(clean_text)
df_train.head()

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,autoimmune diseases tend come clusters gilenya...,gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,completely understand want try results reporte...,gilenya,2
2,fe809672251f6bd0d986e00380f48d047c7e7b76,interesting targets s1p 1 5 receptors rather 1...,fingolimod,2
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,interesting grand merci wonder lemtrada ocrevu...,ocrevus,2
4,b227688381f9b25e5b65109dd00f7f895e838249,hi everybody latest mri results brain cervical...,gilenya,1


In [42]:
df_train['text'].values[0]

'autoimmune diseases tend come clusters gilenya feel good think change anything waste time energy taking tysabri feel amazing symptoms dodgy color vision since always know know last month year decade ive decided enjoy ride point worrying'

In [0]:

MAX_NB_WORDS = 50000  # The maximum number of words to be used. (most frequent)
MAX_SEQUENCE_LENGTH = 350
EMBEDDING_DIM = 250
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(np.append(df_train["text"].values, df_test["text"].values))
word_index = tokenizer.word_index


In [12]:
maxwords=len(word_index)+1
maxwords

47616

In [13]:
train_seq = tokenizer.texts_to_sequences(df_train['text'].values)
test_seq = tokenizer.texts_to_sequences(df_test['text'].values)
train_seq = pad_sequences(train_seq, maxlen=MAX_SEQUENCE_LENGTH)
test_seq = pad_sequences(test_seq, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', train_seq.shape)
print('Shape of data tensor:',train_seq[0])

Shape of data tensor: (5279, 350)
Shape of data tensor: [    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0  

In [14]:
vectorizer = CountVectorizer()
train_state=vectorizer.fit_transform(df_train['drug'])
test_state=vectorizer.transform(df_test['drug'])
print(train_state.shape)

(5279, 110)


In [15]:
Y = pd.get_dummies(df_train['sentiment']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (5279, 3)


In [16]:
train_state.shape[1]

110

In [0]:
def LSTM_model1():
    embedding_layer = Embedding(MAX_NB_WORDS,
                                300,
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True)
    dropout = SpatialDropout1D(0.2)
    mask_layer = Masking()
    lstm_layer = LSTM(128,recurrent_dropout=0.2)
    seq_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
    dense_input = Input(shape=(train_state.shape[1],))
    dense_vector = BatchNormalization()(dense_input)
    phrase_vector = lstm_layer(mask_layer(dropout(embedding_layer(seq_input))))
    feature_vector = concatenate([phrase_vector, dense_vector])
    feature_vector = Dense(64, activation="relu")(feature_vector)
    output = Dense(3, activation="softmax")(feature_vector)
    model = Model(inputs=[seq_input, dense_input], outputs=output)
    return model

def CNN_model():
    CNN_model= Sequential()
    CNN_model.add(Embedding(MAX_NB_WORDS,300,input_length=MAX_SEQUENCE_LENGTH))
    CNN_model.add(Dropout(0.2))
    CNN_model.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
    CNN_model.add(GlobalMaxPooling1D())
    CNN_model.add(Dense(128,activation='relu'))
    CNN_model.add(Dropout(0.2))
    CNN_model.add(Dense(3,activation='softmax'))
    CNN_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return CNN_model
  
def GRU_model():
  model = Sequential()
  model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
  model.add(SpatialDropout1D(0.2))
  model.add(Bidirectional(CuDNNGRU(128,)))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(3, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [0]:

def split_train_test(data, test_ratio=0.15):
  np.random.seed(42)
  shuffled_indices = np.random.permutation(data.shape[0])
  test_set_size = int((data.shape[0]) * test_ratio)
  test_indices = shuffled_indices[:test_set_size]
  train_indices = shuffled_indices[test_set_size:]
  return data[train_indices], data[test_indices]



In [0]:
X_train_seq, X_val_seq=split_train_test(train_seq)
y_train, y_val=split_train_test(Y)
X_train_state, X_val_state= split_train_test(train_state)

**DEEP LEARNING APPROACH**

In [21]:
    model1 = LSTM_model1()
    model1.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])
    
    early_stopping = EarlyStopping(monitor="val_acc", patience=3, verbose=1)
    
    print("Training the model...")
    model1.fit([X_train_seq, X_train_state], y_train, validation_data=([X_val_seq, X_val_state], y_val),
              epochs=10, batch_size=64, callbacks=[early_stopping], verbose=1)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Training the model...
Train on 4488 samples, validate on 791 samples
Epoch 1/10
4488/4488 [==============================] - 72s 16ms/step - loss: 0.8010 - acc: 0.7068 - val_loss: 0.7853 - val_acc: 0.7155
Epoch 2/10
4488/4488 [==============================] - 66s 15ms/step - loss: 0.6444 - acc: 0.7442 - val_loss: 0.7812 - val_acc: 0.6966
Epoch 3/10
4488/4488 [==============================] - 66s 15ms/step - loss: 0.4206 - acc: 0.8425 - val_loss: 0.8798 - val_acc: 0.6852
Epoch 4/10
4488/4488 [==============================] - 66s 15ms/step - loss: 0.2357 - acc: 0.9249 - val_loss: 1.0231 - val_acc: 0.6523
Epoch 00004: early stopping


In [22]:
    model2 = CNN_model()
    sgd = optimizers.adam(lr=0.001)
    model2.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["acc"])
    
    early_stopping = EarlyStopping(monitor="val_acc", patience=5, verbose=1)
    
    print("Training the model...")
    model2.fit(X_train_seq, y_train, validation_data=(X_val_seq, y_val),
              epochs=10, batch_size=64, callbacks=[early_stopping], verbose=1)

Training the model...
Train on 4488 samples, validate on 791 samples
Epoch 1/10
4488/4488 [==============================] - 5s 1ms/step - loss: 0.8079 - acc: 0.7003 - val_loss: 0.7718 - val_acc: 0.7168
Epoch 2/10
4488/4488 [==============================] - 3s 561us/step - loss: 0.7147 - acc: 0.7259 - val_loss: 0.7562 - val_acc: 0.7168
Epoch 3/10
4488/4488 [==============================] - 3s 562us/step - loss: 0.5479 - acc: 0.7636 - val_loss: 0.7631 - val_acc: 0.7105
Epoch 4/10
4488/4488 [==============================] - 3s 561us/step - loss: 0.2331 - acc: 0.9372 - val_loss: 0.8795 - val_acc: 0.6827
Epoch 5/10
4488/4488 [==============================] - 3s 561us/step - loss: 0.0744 - acc: 0.9889 - val_loss: 0.9670 - val_acc: 0.6802
Epoch 6/10
4488/4488 [==============================] - 3s 559us/step - loss: 0.0401 - acc: 0.9953 - val_loss: 1.1591 - val_acc: 0.7042
Epoch 00006: early stopping


In [23]:
    model3 = GRU_model()
    model3.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])
    
    early_stopping = EarlyStopping(monitor="val_acc", patience=3, verbose=1)
    
    print("Training the model...")
    model3.fit(X_train_seq, y_train, validation_data=(X_val_seq, y_val),
              epochs=10, batch_size=8, callbacks=[early_stopping], verbose=1)

Training the model...
Train on 4488 samples, validate on 791 samples
Epoch 1/10
4488/4488 [==============================] - 68s 15ms/step - loss: 0.7760 - acc: 0.7248 - val_loss: 0.7742 - val_acc: 0.7130
Epoch 2/10
4488/4488 [==============================] - 67s 15ms/step - loss: 0.5930 - acc: 0.7674 - val_loss: 0.8766 - val_acc: 0.7004
Epoch 3/10
4488/4488 [==============================] - 67s 15ms/step - loss: 0.3299 - acc: 0.8817 - val_loss: 0.9904 - val_acc: 0.6473
Epoch 4/10
4488/4488 [==============================] - 67s 15ms/step - loss: 0.1594 - acc: 0.9485 - val_loss: 1.2296 - val_acc: 0.6435
Epoch 00004: early stopping


In [24]:

pred1 = model1.predict([X_val_seq,X_val_state])
print('f1_score for {} is: {:0.3f}'.format('lstm',f1_score(np.argmax(y_val,axis=1),np.argmax(pred1,axis=1), average='macro')))
pred2 = model2.predict(X_val_seq)
print('f1_score for {} is: {:0.3f}'.format('cnn',f1_score(np.argmax(y_val,axis=1),np.argmax(pred2,axis=1), average='macro')))
pred3 = model3.predict(X_val_seq)
print('f1_score for {} is: {:0.3f}'.format('gru',f1_score(np.argmax(y_val,axis=1),np.argmax(pred3,axis=1), average='macro')))

f1_score for lstm is: 0.440
f1_score for cnn is: 0.425
f1_score for gru is: 0.413


**MACHINE LEARNING APPROACH**

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier

In [0]:
vectorizer = TfidfVectorizer(ngram_range=(1,3),stop_words="english", analyzer='word',max_features=50000)
X =vectorizer.fit_transform(df_train['text'])
X_test =vectorizer.transform(df_test['text'])
Y =df_train['sentiment']

In [0]:
X_train,X_val,y_train,y_val = train_test_split(X,Y,test_size=0.15,random_state =0)

In [31]:

C=np.logspace(-4,2,6)
gamma=np.logspace(-4,2,6)
pipe=Pipeline([("classifier",RandomForestClassifier())])
param_grid=[{"classifier":[RandomForestClassifier()],
            "classifier__n_estimators":[10,100,1000],
            "classifier__max_features":[1,2,3]},
           {"classifier":[OneVsRestClassifier(LogisticRegression(class_weight='balanced'))],
            "classifier__estimator__penalty":['l1','l2'],
            "classifier__estimator__C":C},
           {"classifier":[SVC()],
            "classifier__gamma":gamma,
            "classifier__C":C},
           {"classifier":[MultinomialNB()]}]
      
score=make_scorer(f1_score, average='macro')

grid=GridSearchCV(pipe,param_grid,cv=5,verbose=1,n_jobs=-1,scoring=score)
grid.fit(X_train,y_train)

print('best score',grid.best_score_)
print('best param:',grid.best_params_)

pred=grid.predict(X_val)
print('f1_score on validation set:',f1_score(y_val,pred,average='macro'))
print('accuracy on validation set:',accuracy_score(y_val,pred))

Fitting 5 folds for each of 58 candidates, totalling 290 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 290 out of 290 | elapsed: 20.4min finished


best score 0.4873237031593002
best param: {'classifier': OneVsRestClassifier(estimator=LogisticRegression(C=0.3981071705534969,
                                                 class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='warn', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None), 'classifier__estimator__C': 0.3981071705534969, 'classifier__estimator__penalty': 'l2'}
f1_score on validation set: 0.52445536415298
accuracy on validation set: 0.6691

In [35]:
test_pred=grid.predict(X_test)
df_test["sentiment"]=test_pred.astype('int')
df_test['sentiment'].value_counts()

2    1939
1     628
0     357
Name: sentiment, dtype: int64

In [0]:
test=pd.read_csv('/content/test_tOlRoBf.csv')
test['sentiment']=df_test['sentiment']
test.to_csv('/content/final_test_tOlRoBf.csv')
